In [ ]:
from ase.visualize import view

from ase import units
from ase.md import MDLogger
from ase.io import Trajectory
from ase.calculators.emt import EMT
from ase.calculators.eam import EAM
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

##### building structure

In [2]:
cu_strc = FaceCenteredCubic(
    directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], symbol="Cu", size=(3, 3, 3), pbc=True
)

# view(cu_strc)

In [3]:
cu_strc

Lattice(symbols='Cu108', pbc=True, cell=[10.83, 10.83, 10.83])

#### Attaching a calculator

In [4]:
cu_strc.calc = EMT()

In [5]:
cu_strc

Lattice(symbols='Cu108', pbc=True, cell=[10.83, 10.83, 10.83], calculator=EMT(...))

##### initial velocity from the maxwell-boltzmann distribution at 300 K

In [6]:
MaxwellBoltzmannDistribution(atoms=cu_strc, temperature_K=300)

In [7]:
cu_strc

Lattice(symbols='Cu108', pbc=True, cell=[10.83, 10.83, 10.83], momenta=..., calculator=EMT(...))

##### NVE simulation using velocity verlet algorithm in ASE

In [10]:
dyn = VelocityVerlet(cu_strc, 5 * units.fs, trajectory='cu_strc.traj')

In [8]:
def printenergy(a):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print(
        f'Energy per atom: Epot ={epot:6.3f}eV  Ekin = {ekin:.3f}eV '
        f'(T={ekin / (1.5 * units.kB):3.0f}K) Etot = {epot + ekin:.3f}eV'
    )

In [11]:
printenergy(cu_strc)
for i in range(30):
    dyn.run(10)
    printenergy(cu_strc)

Energy per atom: Epot =-0.006eV  Ekin = 0.040eV (T=308K) Etot = 0.034eV
Energy per atom: Epot = 0.025eV  Ekin = 0.009eV (T= 68K) Etot = 0.034eV
Energy per atom: Epot = 0.014eV  Ekin = 0.020eV (T=156K) Etot = 0.034eV
Energy per atom: Epot = 0.015eV  Ekin = 0.019eV (T=148K) Etot = 0.034eV
Energy per atom: Epot = 0.011eV  Ekin = 0.023eV (T=179K) Etot = 0.034eV
Energy per atom: Epot = 0.014eV  Ekin = 0.020eV (T=154K) Etot = 0.034eV
Energy per atom: Epot = 0.017eV  Ekin = 0.017eV (T=131K) Etot = 0.034eV
Energy per atom: Epot = 0.011eV  Ekin = 0.023eV (T=178K) Etot = 0.034eV
Energy per atom: Epot = 0.018eV  Ekin = 0.016eV (T=125K) Etot = 0.034eV
Energy per atom: Epot = 0.010eV  Ekin = 0.025eV (T=190K) Etot = 0.034eV
Energy per atom: Epot = 0.016eV  Ekin = 0.018eV (T=138K) Etot = 0.034eV
Energy per atom: Epot = 0.015eV  Ekin = 0.019eV (T=148K) Etot = 0.034eV
Energy per atom: Epot = 0.016eV  Ekin = 0.018eV (T=142K) Etot = 0.034eV
Energy per atom: Epot = 0.009eV  Ekin = 0.026eV (T=198K) Etot = 

##### note: working wiht ASE trajectory file

In [4]:
from ase.io.trajectory import read_traj, Trajectory

In [13]:
traj = Trajectory('cu_strc.traj')

view(traj)

<Popen: returncode: None args: ['c:\\Users\\Acer\\anaconda3\\python.exe', '-...>

##### Considering ASE based Logger (MDLogger)

In [17]:
cu_strc2 = FaceCenteredCubic(
    directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], symbol="Cu", size=(3, 3, 3), pbc=True
)

cu_strc2.calc = EMT()


In [18]:
MaxwellBoltzmannDistribution(atoms=cu_strc2, temperature_K=300)

In [16]:
# cu_strc2.get_momenta()

In [19]:
dyn2 = VelocityVerlet(cu_strc2, 5 * units.fs, trajectory='cu_strc2.traj')

In [ ]:
md_logger_1 =  MDLogger(dyn=dyn2, atoms=cu_strc2, logfile='mdlog_cu_strc2.log', header=True, stress=True, peratom=True, mode="w")

dyn2.attach(md_logger_1, interval=10)

dyn2.run(1000)

True

#### NVT

###### Langevin

In [34]:
cu_strc3 = FaceCenteredCubic(
    directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], symbol="Cu", size=(3, 3, 3), pbc=True
)

cu_strc3.calc = EMT()

In [35]:
MaxwellBoltzmannDistribution(atoms=cu_strc3, temperature_K=300)

In [36]:
dyn3 =  Langevin(cu_strc3, 5 * units.fs, temperature_K=300, friction=0.02)

In [37]:
md_logger_2 =  MDLogger(dyn=dyn3, atoms=cu_strc3, logfile='mdlog_cu_strc3.log', header=True, stress=True, peratom=True, mode="w")
traj3 = Trajectory('cu_strc3.traj', 'w', cu_strc3)

In [38]:
dyn3.attach(md_logger_2, interval=10)
dyn3.attach(traj3, interval=10)

In [39]:
dyn3.run(1000)

True

In [ ]:
cu_strc3.calc = EAM

ValueError: invalid literal for int() with base 10: 'Cmax(1,1,1)'